In [ ]:
import sqlite3
import numpy as np

In [ ]:
!ls

**Question 1:** Which team scored the most points when playing at home?

In [ ]:
conn = sqlite3.connect('database.sqlite')

In [ ]:
c = conn.cursor()

In [ ]:
conn

In [ ]:
list(c.execute('''SELECT SUM(m.home_team_goal), t.team_long_name
                  FROM Match m
                  JOIN Team t
                  ON team_api_id = home_team_api_id
                  GROUP BY 2
                  ORDER BY 1 DESC
                  LIMIT 1;'''))

Real Madrid CF scored the most goals at home. They scored 505 goals at home.

**Question 2:** Did this team also score the most points when playing away?

In [ ]:
list(c.execute('''SELECT SUM(m.away_team_goal), t.team_long_name
                  FROM Match m
                  JOIN Team t
                  ON team_api_id = away_team_api_id
                  GROUP BY 2
                  ORDER BY 1 DESC
                  LIMIT 1;'''))

**Answer:** No. FC Barcelona scored the most points while away.

**Question 3:** How many matches resulted in a tie?

In [ ]:
ties = list(c.execute('''SELECT COUNT(*)
                  FROM Match m
                  WHERE m.home_team_goal = m.away_team_goal;'''))

ties

**Answer:** 6596 matches ended in ties.

**Question 4:** How many players have Smith for their last name? How many have 'smith' anywhere in their name?

In [ ]:
names = list(c.execute('''SELECT COUNT(*)
                          FROM Player pa
                          WHERE pa.player_name LIKE '%Smith' '''))

In [ ]:
names_anywhere = list(c.execute('''SELECT COUNT(*)
                          FROM Player pa
                          WHERE pa.player_name LIKE '%Smith%' '''))

In [ ]:
print(names)
print(names_anywhere)

**Answer:** 18 for both.

**Question 5:** What was the median tie score? Use the value determined in the previous question for the number of tie games. Hint: PostgreSQL does not have a median function. Instead, think about the steps required to calculate a median and use the WITH command to store stepwise results as a table and then operate on these results.

In [ ]:
tie_scores = list(c.execute('''WITH t1 AS (SELECT COUNT(*)
                               FROM Match m
                               WHERE m.home_team_goal = m.away_team_goal),
                               
                               t2 AS (SELECT m.home_team_goal AS score
                               FROM Match m
                               WHERE m.home_team_goal = m.away_team_goal
                               ORDER BY 1 ASC
                               LIMIT (SELECT * FROM t1)/2)
                               
                               SELECT t2.score
                               FROM t2
                               ORDER BY 1 DESC 
                               LIMIT 1'''))

In [ ]:
tie_scores[0][0]

Median tie score is 1.

**Question 6:** What percentage of players prefer their left or right foot? Hint: Calculate either the right or left foot, whichever is easier based on how you setup the problem.

In [ ]:
list(c.execute('''
                SELECT 
                FROM player_attributes
               '''))

In [ ]:
list(c.execute('''
                  WITH t1 AS (SELECT DISTINCT player_fifa_api_id, CASE preferred_foot WHEN 'right' THEN 1
                              WHEN 'left' THEN 0
                              END AS foot
                              FROM player_attributes
                              WHERE preferred_foot IS NOT NULL)
                    
                  SELECT AVG(t1.foot)
                  FROM t1 
                '''))

73.7% of players are right footed.